In [ ]:
from PIL import Image
import piexif
import os
import pandas as pd

ORIGINAL_LIBRARY_DIR_NAME = "original-photos"
PROCESSED_LIBRARY_DIR_NAME = "processed-photos"
PROCESSED_LIBRARY_DIR = os.path.join(PROCESSED_LIBRARY_DIR_NAME)

os.makedirs(PROCESSED_LIBRARY_DIR, exist_ok=True)

codec = "ISO-8859-1"  # or latin-1


def exif_to_tag(exif_dict):
    exif_tag_dict = {}
    thumbnail = exif_dict.pop("thumbnail")
    if thumbnail:
        exif_tag_dict["thumbnail"] = thumbnail.decode(codec)

    for ifd in exif_dict:
        exif_tag_dict[ifd] = {}
        for tag in exif_dict[ifd]:
            try:
                element = exif_dict[ifd][tag].decode(codec)

            except AttributeError:
                element = exif_dict[ifd][tag]

            exif_tag_dict[ifd][piexif.TAGS[ifd][tag]["name"]] = element

    return exif_tag_dict


def find_all_images() -> list[str]:
    """Performs recursive tree search of the original library directory and returns a list of all images found."""
    images = []

    for root, dirs, files in os.walk(ORIGINAL_LIBRARY_DIR_NAME):
        for file in files:
            if file.lower().endswith(
                (".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".heic")
            ):
                images.append(os.path.join(root, file))

    return images


def get_image_metadata(image_path) -> dict | None:
    """Extracts and returns all metadata from an image file."""
    try:
        img = Image.open(image_path)
        exif_data = piexif.load(img.info["exif"])
        if exif_data:
            tags = exif_to_tag(exif_data)
            flattened_tags = pd.json_normalize(tags, sep="_").to_dict(orient="records")[
                0
            ]
            flattened_tags["image_path"] = image_path
            return flattened_tags

        else:
            return None

    except (KeyError, piexif.InvalidImageDataError):
        return None


def process_library():
    image_paths = []
    for image_path in find_all_images():
        image_paths.append(image_path)

    metadata_list = []

    for image_path in image_paths:
        metadata = get_image_metadata(image_path)
        if metadata:
            metadata_list.append(metadata)

    return metadata_list

In [48]:
metadata = process_library()
metadata_df = pd.DataFrame(metadata).set_index("image_path")
metadata_df.to_csv("image_metadata.csv")
metadata_df

,0th_NewSubfileType,0th_ImageDescription,0th_Make,0th_Model,0th_Orientation,0th_XResolution,0th_YResolution,0th_ResolutionUnit,0th_Software,0th_DateTime,...,Exif_SceneCaptureType,Exif_GainControl,Exif_Contrast,Exif_Saturation,Exif_Sharpness,Exif_SubjectDistanceRange,0th_YCbCrPositioning,Exif_ExifVersion,Exif_ComponentsConfiguration,Exif_FlashpixVersion
image_path,,,,,,,,,,,,,,,,,,,,,
original-photos/20260120_0043.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:20 23:06:16,...,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
original-photos/20260120_0026.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:20 23:06:07,...,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
original-photos/20260109_0002.JPEG,NaN,NaN,NaN,NaN,1,"(72, 1)","(72, 1)",2,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,1.0,0221, ,0100
original-photos/20260117_0001.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:17 14:12:43,...,0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
original-photos/20260108_0007.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:08 15:59:27,...,0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
original-photos/20260108_0008.JPEG,NaN,NaN,NIKON CORPORATION,NIKON D3300,1,"(72, 1)","(72, 1)",2,darktable 4.6.1,2026:01:08 15:59:27,...,0,1.0,0.0,0.0,0.0,0.0,1.0,0221, ,0100
original-photos/20260117_0002.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:17 14:12:44,...,0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
original-photos/20260120_0014.jpg,1.0,,NIKON CORPORATION,NIKON D3300,1,"(300, 1)","(300, 1)",2,darktable 4.6.1,2026:01:20 12:18:05,...,0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
